In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import pickle


def get_metrics(model, axis, preprocessed_test=None):
    prob_preds = model.predict_proba(x_train)
    performance = roc_auc_score(y_train, prob_preds[:, axis])
    print "TRAINING: " + str(performance)
    
    prob_preds = model.predict_proba(x_test)
    performance = roc_auc_score(y_test, prob_preds[:, axis])
    print "TEST: " + str(performance)

/anaconda/lib/python2.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
x = np.load("data/x_normalized.npy")
y = np.load("data/y.npy")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
model = lgb.LGBMClassifier(boosting_type='dart',learning_rate=0.1, num_leaves=500, reg_alpha=0, min_child_samples=5, min_child_weight=1e-05, n_estimators=1000, reg_lambda=100, max_depth=15, dropout=0.7)
model.fit(x_train, y_train, verbose=False)

y_pred = model.predict_proba(x_train)[:, 1]
performance = roc_auc_score(y_train, y_pred)
print performance

y_pred = model.predict_proba(x_test)[:, 1]
performance = roc_auc_score(y_test, y_pred)
print performance

0.9881389025222007
0.7976760596588771


In [4]:
from xgboost import XGBClassifier

model1 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=1, learning_rate=0.05, max_delta_step=0,
       max_depth=10, min_child_weight=10, missing=None, n_estimators=200,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)
model1.fit(x_train, y_train)

y_pred = model1.predict_proba(x_train)[:, 1]
performance = roc_auc_score(y_train, y_pred)
print performance

y_pred = model1.predict_proba(x_test)[:, 1]
performance = roc_auc_score(y_test, y_pred)
print performance

0.9104873405681265
0.7978625901336096


In [5]:
xgboost_predictions = model.predict_proba(x_train)[:, 1]
lgb_predictions = model1.predict_proba(x_train)[:, 1]

In [6]:
xgboost_predictions

array([0.14159794, 0.42136143, 0.08245563, ..., 0.19948283, 0.62061479,
       0.05624542])

In [7]:
lgb_predictions

array([0.2386743 , 0.38075927, 0.12050674, ..., 0.24197845, 0.59291583,
       0.09094482], dtype=float32)

In [8]:
xgboost_predictions = xgboost_predictions.reshape(-1,1)
lgb_predictions = lgb_predictions.reshape(-1, 1)

In [9]:
new_x_train = np.concatenate((xgboost_predictions, lgb_predictions), axis=1)
new_x_train.shape

(51733, 2)

In [18]:
xgboost_predictions = model.predict_proba(x_test)[:, 1]
lgb_predictions = model1.predict_proba(x_test)[:, 1]

In [19]:
xgboost_predictions = xgboost_predictions.reshape(-1,1)
lgb_predictions = lgb_predictions.reshape(-1, 1)

In [20]:
new_x_test = np.concatenate((xgboost_predictions, lgb_predictions), axis=1)

In [21]:
new_x_test.shape

(12934, 2)

In [22]:
np.save("hehe/new_x_train.npy", new_x_train)
np.save("hehe/y_train.npy", y_train)
np.save("hehe/new_x_test.npy", new_x_test)
np.save("hehe/y_test.npy", y_test)

In [17]:
pkl_filename = "hehe/best_xgboost.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(model, file)
    
pkl_filename = "hehe/best_lgb.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(model1, file)